# דוגמה בסיסית של AutoGen

בדוגמה קוד זו, תשתמשו במסגרת ה-AI [AutoGen](https://aka.ms/ai-agents/autogen) כדי ליצור סוכן בסיסי.

המטרה של דוגמה זו היא להציג את השלבים שבהם נשתמש בהמשך בדוגמאות קוד נוספות בעת יישום דפוסי סוכנות שונים.


## ייבוא חבילות הפייתון הנדרשות


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## יצירת הלקוח

בדוגמה זו, נשתמש ב-[GitHub Models](https://aka.ms/ai-agents-beginners/github-models) לצורך גישה ל-LLM.

ה-`model` מוגדר כ-`gpt-4o-mini`. נסו לשנות את המודל למודל אחר הזמין בשוק של GitHub Models כדי לראות תוצאות שונות.

כמבחן מהיר, פשוט נריץ שאלה פשוטה - `מהי בירת צרפת`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## הגדרת הסוכן

עכשיו, לאחר שהגדרנו את `client` ואישרנו שהוא פועל, בואו ניצור `AssistantAgent`. לכל סוכן ניתן להקצות:
**name** - שם קצר שיהיה שימושי להתייחסות אליו בתהליכים מרובי סוכנים.  
**model_client** - הלקוח שיצרתם בשלב הקודם.  
**tools** - כלים זמינים שהסוכן יכול להשתמש בהם כדי להשלים משימה.  
**system_message** - ההנחיה שמגדירה את המשימה, ההתנהגות והטון של ה-LLM.  

ניתן לשנות את הודעת המערכת כדי לראות כיצד ה-LLM מגיב. נעסוק ב-`tools` בשיעור מספר 4.  


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## הפעלת הסוכן

הפונקציה הבאה תפעיל את הסוכן. אנו משתמשים בשיטה `on_message` כדי לעדכן את מצב הסוכן עם ההודעה החדשה.

במקרה הזה, אנו מעדכנים את המצב עם הודעה חדשה מהמשתמש, שהיא `"תכנן לי חופשה שמשית נהדרת"`.

ניתן לשנות את תוכן ההודעה כדי לראות כיצד ה-LLM מגיב באופן שונה.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש להיות מודעים לכך שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
